<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/8.Machine_Learning/20_ejercicio_integrador_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio integrador

El objetivo del siguiente ejercicio es aplicar los conocimientos obtenidos sobre los algoritmos más utilizados en aprendizaje supervisado.

Para ello se les brinda un dataset al cual deberán realizar las transformaciones necesarias, definir la variable objetivo, tipo de problema (Regresión/Clasificación) y entrenar un modelo en base al problema.

**Dataset**: `imdb_dataset.csv`

https://drive.google.com/file/d/1IxHx3ogmQ_Zy1gvg2Bz1lZDdb-wSfbeY/view?usp=sharing

## Importar Librerias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from imblearn.under_sampling import RandomUnderSampler
warnings.filterwarnings("ignore")


# nltk

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer


#TODO sklearn libraries - Completar con el/los modelos predictivos

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn import preprocessing, svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

#TODO modelos para comparar

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression






In [ ]:
nltk.download('stopwords')

## Carga de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/20231030/imdb_dataset.csv') #TODO
df.head()

## Data

Para este ejercicio nos limitamos a tomar solo un subset de `10000` instancias.

In [ ]:
df_positive = df[df['sentiment']=='positive'][:9000]
df_negative = df[df['sentiment']=='negative'][:1000]

df_review_imb = pd.concat([df_positive,df_negative ])

In [ ]:
colors = sns.color_palette('deep')

plt.figure(figsize=(8,4), tight_layout=True)
plt.bar(x=['Positive', 'Negative'],
        height=df_review_imb.value_counts(['sentiment']),
        color=colors[:2])
plt.title('Sentiment')
plt.savefig('sentiment.png')
plt.show()

## Preparación y análisis de datos (EDA)

## Resample data

El desequilibrio de datos es un gran problema para las tareas de clasificación. En Python, existe una biblioteca que permite el uso de muchos algoritmos para manejar este estado desequilibrado de los datos y sus daños.

imbalanced-learn es un paquete de Python que ofrece varias técnicas de remuestreo comúnmente utilizadas en conjuntos de datos que muestran un fuerte desequilibrio entre clases. Es compatible con scikit-learn y forma parte de proyectos `scikit-learn`.

Para volver a muestrear nuestros datos utilizamos la biblioteca `imblearn`. Puede submuestrear reseñas positivas o sobremuestrear reseñas negativas (debe elegir según los datos con los que está trabajando). En este caso, usaremos `RandomUnderSampler`

In [ ]:

rus = RandomUnderSampler(random_state= 0)
df_review_bal,df_review_bal['sentiment']=rus.fit_resample(df_review_imb[['review']],
                                                          df_review_imb['sentiment'])


df_review_bal

In [ ]:
print(df_review_imb.value_counts('sentiment'))
print(df_review_bal.value_counts('sentiment'))

## Tokenization
La tokenización es una forma de separar un fragmento de texto en unidades más pequeñas llamadas tokens. Aquí, los tokens pueden ser palabras, caracteres o subpalabras. Por lo tanto, la tokenización se puede clasificar en términos generales en 3 tipos: tokenización de palabras, caracteres y subpalabras (caracteres de n-gramas).

Por ejemplo, considere la frase: "Nunca te rindas".

La forma más común de formar tokens se basa en el espacio. Asumiendo el espacio como delimitador, la tokenización de la oración da como resultado 3 tokens: Nunca te rindas. Como cada token es una palabra, se convierte en un ejemplo de tokenización de Word.

In [ ]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
#Funciones para limpieza de datos

def clean_data(text):          #using the re library

    text = text.lower().strip()
    text = re.sub("^a-zA-z0-9\s","", text)
    text = re.sub(r"<br>", " ", text)
    text = re.sub(r"([-?.!,/\"])", r" \1 ", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,']", "", text)
    text = re.sub(r"[ ]+", " ", text)
    text = text.rstrip().strip()

    return text

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [ ]:
#Apply clean data
df_review_bal['review'] = df_review_bal.review.apply(clean_data)

#Apply function on review column
df_review_bal['review']=df_review_bal['review'].apply(denoise_text)

df_review_bal.head()

## Stemming text

La derivación es una técnica que se utiliza para reducir una palabra flexionada hasta la raíz de la palabra. Por ejemplo, las palabras `programación`, `programador` y `programas` se pueden reducir a la raíz común de la palabra `programa`. En otras palabras, `programa` se puede utilizar como sinónimo de las tres palabras de inflexión anteriores.

In [ ]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

In [ ]:
#Apply function on review column
df_review_bal['review']=df_review_bal['review'].apply(simple_stemmer)

## Stopwords

`Stopwords` una palabra de parada de uso común (como “el”, “a”, “una”, “en”) que un motor de búsqueda ha sido programado para ignorar, tanto al indexar entradas para la búsqueda como al recuperarlas. como resultado de una consulta de búsqueda.

No queremos que estas palabras ocupen espacio en nuestra base de datos ni que consuman un tiempo de procesamiento valioso. Para ello, podemos eliminarlas fácilmente, almacenando una lista de palabras que consideres vacías.

In [ ]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [ ]:
#Apply function on review column
df_review_bal['review']=df_review_bal['review'].apply(remove_stopwords)

## Data split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_review_bal['review'],
                                                                df_review_bal['sentiment'],
                                                                test_size=0.2,
                                                                random_state=42)


In [ ]:
X_train

In [ ]:
y_train.value_counts()

## Text Representation (Bag of words)

Los clasificadores y algoritmos de aprendizaje esperan vectores de características numéricas en lugar de documentos de texto sin formato. Es por eso que necesitamos convertir el texto de reseñas de películas en vectores numéricos.

usaremos bolsa de palabras (BOW) ya que nos importa la frecuencia de las palabras en las revisiones de texto; sin embargo, el orden de las palabras es irrelevante. Dos formas comunes de representar una bolsa de palabras son CountVectorizer y Term Frequency, Inverse Document Frequency (TF-IDF).

Queremos identificar palabras únicas/representativas para reseñas positivas y negativas, por lo que elegiremos TF-IDF. Para convertir datos de texto en vectores numéricos con TF-IDF

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(X_train)
# also fit the test_x_vector
test_x_vector = tfidf.transform(X_test)

In [ ]:
train_x_vector.shape

## Entrenamiento modelo definitivo

In [ ]:
# Implementamos SVC
svc = SVC(kernel='linear')
svc.fit(train_x_vector, y_train)

In [ ]:
# Testeamos el modelo
print(svc.predict(tfidf.transform(['A good movie'])))
print(svc.predict(tfidf.transform(['An excellent movie'])))
print(svc.predict(tfidf.transform(['I did not like this movie at all I gave this movie away'])))

In [ ]:
# Implementamos LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(train_x_vector,y_train)

In [ ]:
# Testeamos el modelo
print(log_reg.predict(tfidf.transform(['A good movie'])))
print(log_reg.predict(tfidf.transform(['An excellent movie'])))
print(log_reg.predict(tfidf.transform(['I did not like this movie at all I gave this movie away'])))

In [ ]:
# Implementamos DecisionTree
dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, y_train)

In [ ]:
# Testeamos el modelo
print(dec_tree.predict(tfidf.transform(['A good movie'])))
print(dec_tree.predict(tfidf.transform(['An excellent movie'])))
print(dec_tree.predict(tfidf.transform(['I did not like this movie at all I gave this movie away'])))

## Evaluación modelo

In [ ]:
print(svc.score(test_x_vector, y_test))
print(log_reg.score(test_x_vector, y_test))
print(dec_tree.score(test_x_vector, y_test))

In [ ]:
y_pred_test = svc.predict(test_x_vector)
print(y_pred_test)

In [ ]:
# 10) Evaluación del modelo sobre el conjunto de test
print(metrics.confusion_matrix(y_test, y_pred_test))